data_load_attempt_2

Looking at:
https://github.com/KeremTurgutlu/myCNN/blob/master/CUSTOM%20CNN%20WITH%20PYTORCH.ipynb
https://github.com/motiur/fastai/blob/master/statoil_4.ipynb

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.model import *
from fastai.dataset import *
from fastai.conv_learner import *

In [3]:
PATH  = "data/"
TRAIN = "train/"
VALID = "valid/"
TEST  = "test/"

In [4]:
train_data = pd.read_json(PATH+TRAIN+'train.json')

In [19]:
train_data_array = []
for idx, row in train_data.iterrows():
    band_1 = row['band_1']
    band_2 = row['band_2']
    stacked = np.dstack([np.array(band_1).reshape(75,75), 
                         np.array(band_2).reshape(75,75),
                         np.array(band_1).reshape(75,75) / np.array(band_2).reshape(75,75),
                        (np.array(band_1).reshape(75,75) + np.array(band_2).reshape(75,75)) / 2])
    train_data_array.append(stacked)
train_data_array = np.array(train_data_array)

In [7]:
print(train_data_array.shape)

(1604, 75, 75, 4)


#### PyTorch Dataset:

In [8]:
# Creating FloatTensors from NumPy arrays
train_data_tensor = torch.FloatTensor(train_data_array[:,:,:,:])

In [9]:
# Creating LongTensor from response of train
train_label_tensor = torch.LongTensor(np.array(train_data.is_iceberg.loc[:].values).astype('int64'))

In [10]:
# Creating TensorDatasets
train_dataset = TensorDataset(data_tensor=train_data_tensor, target_tensor=train_label_tensor)

In [11]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, 
                                          shuffle=True, num_workers=4)

In [12]:
dataloaders = {'train':trainloader, 'val':trainloader}
datasizes   = {'train':train_data_tensor.size(0), 'val':train_data_tensor.size(0)}

#### MODEL

In [13]:
ARCH = resnet34

In [20]:
train_data_tensor = train_data_array

In [21]:
resnet_data = ImageClassifierData.from_arrays(PATH, (train_data_tensor, train_label_tensor),
                                              (train_data_tensor, train_label_tensor), bs=16)

In [22]:
learn = ConvLearner.pretrained(ARCH, resnet_data, precompute=True)

  0%|          | 0/101 [00:00<?, ?it/s]


RuntimeError: Input type (Double) and weight type (Float) should be the same